In [ ]:
# !pip install -Uqqq pip --progress-bar off
!pip install -q runpod
!pip install -q text-generation
!pip install -q requests

```json
{
   "data" : {
      "gpuTypes" : [
         ...
         {
            "displayName" : "RTX 3090",
            "id" : "NVIDIA GeForce RTX 3090",
            "memoryInGb" : 24
         },
         {
            "displayName" : "RTX 3090 Ti",
            "id" : "NVIDIA GeForce RTX 3090 Ti",
            "memoryInGb" : 24
         },
         {
            "displayName" : "RTX 4090",
            "id" : "NVIDIA GeForce RTX 4090",
            "memoryInGb" : 24
         },
         {
            "displayName" : "RTX A4500",
            "id" : "NVIDIA RTX A4500",
            "memoryInGb" : 20
         },
         {
            "displayName" : "RTX A5000",
            "id" : "NVIDIA RTX A5000",
            "memoryInGb" : 24
         },
         {
            "displayName" : "RTX A6000",
            "id" : "NVIDIA RTX A6000",
            "memoryInGb" : 48
         }
         ...
      ]
   }
}
```

In [9]:
import requests
import runpod
from text_generation import Client

ModuleNotFoundError: No module named 'runpod'

In [57]:
runpod.api_key = "YOUR RUNPOD ID"

In [58]:
gpu_count = 1

pod = runpod.create_pod(
    name="Llama-7b-chat",
    image_name="ghcr.io/huggingface/text-generation-inference:0.9.4",
    gpu_type_id="NVIDIA RTX A4500",
    data_center_id="EU-RO-1",
    cloud_type="SECURE",
    docker_args="--model-id TheBloke/Llama-2-7b-chat-fp16",
    gpu_count=gpu_count,
    volume_in_gb=50,
    container_disk_in_gb=5,
    ports="80/http,29500/http",
    volume_mount_path="/data",
)
pod

{'id': 'c15whkqgi56r2g',
 'imageName': 'ghcr.io/huggingface/text-generation-inference:0.9.4',
 'env': [],
 'machineId': 'rhfeb111mf0v',
 'machine': {'podHostId': 'c15whkqgi56r2g-64410ed2'}}

In [59]:
SERVER_URL = f'https://{pod["id"]}-80.proxy.runpod.net'
print(SERVER_URL)

https://c15whkqgi56r2g-80.proxy.runpod.net


In [60]:
print(f"Docs (Swagger UI) URL: {SERVER_URL}/docs")

Docs (Swagger UI) URL: https://c15whkqgi56r2g-80.proxy.runpod.net/docs


In [81]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

## API

In [82]:
def make_request(prompt: str):
    data = {
        "inputs": prompt,
        "parameters": {"best_of": 1, "temperature": 0.01, "max_new_tokens": 512},
    }
    headers = {"Content-Type": "application/json"}

    return requests.post(f"{SERVER_URL}/generate", json=data, headers=headers)

In [83]:
%%time
prompt = generate_prompt(
    "Write an email to a new client to offer a subscription for a paper supply for 1 year."
)
response = make_request(prompt)

CPU times: user 105 ms, sys: 3.15 ms, total: 108 ms
Wall time: 11 s


In [84]:
response.status_code

200

In [86]:
print(response.json()["generated_text"].strip())

Subject: Welcome to [Company Name] - Paper Supply Subscription Offer
Dear [Client Name],
We are thrilled to welcome you to [Company Name], and we hope you're doing well! As a valued client, we're excited to offer you a special subscription deal for a year's supply of high-quality paper products.
Our paper supply subscription service is designed to provide you with a convenient and cost-effective way to stock up on the paper products you need, without any hassle or waste. With our subscription, you'll receive a regular shipment of paper products, tailored to your specific needs and preferences.
Here's what you can expect with our subscription service:
* A wide range of paper products, including A4, A3, A2, A1, and custom sizes
* High-quality, durable paper that's perfect for printing, writing, and crafting
* Regular shipments every [insert time frame, e.g., monthly, quarterly, etc.]
* Flexible subscription plans to suit your needs and budget
* Easy online management and tracking of your

In [93]:
DWIGHT_SYSTEM_PROMPT = """
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information. Always format your responses using markdown.
""".strip()

In [100]:
%%time
prompt = generate_prompt(
    "Write an email to a new client to offer a subscription for a paper supply for 1 year.",
    system_prompt=DWIGHT_SYSTEM_PROMPT,
)
response = make_request(prompt)

CPU times: user 117 ms, sys: 6.58 ms, total: 123 ms
Wall time: 14 s


In [102]:
print(response.json()["generated_text"].strip())

Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive 1-year subscription to our top-notch paper supply! 📝💪 With this subscription, you'll receive a steady stream of premium paper products, carefully curated to meet your every office need. 📈
🌟 What's included in the subscription, you ask? 🤔 Well, let me tell you! Here are just a few of the beet-tastic perks you can look forward to:
🌱 High-quality paper products, including copy paper, printer paper, and even some specialty paper for those extra-special occasions. 🎉
📈 Regular deliveries to ensure a steady supply of paper, so you never have to worry about running out. 😅
📊 A personalized dashboard to track your usage and manage your subscription, so you can stay on top o

## Client

In [103]:
client = Client(SERVER_URL, timeout=60)

In [104]:
%%time
response = client.generate(prompt, max_new_tokens=512).generated_text

CPU times: user 125 ms, sys: 6.81 ms, total: 132 ms
Wall time: 14.3 s


In [106]:
print(response.strip())

Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive opportunity to subscribe to our top-notch paper supply for the next 1 year! 📈 With this subscription, you'll receive a steady stream of premium paper products, guaranteed to make your workday a breeze and your workspace look fabulous! 💪
Here's what you can expect from our Beet-y Awesome Paper Supply Subscription:
🌟 High-quality paper products, carefully selected to meet your every need.
📦 Regular deliveries of paper, so you'll never run out.
📊 A 10% discount on all paper purchases, just for subscribers! 💰
📝 A complimentary Dunder Mifflin pen, just for signing up! 🖋️
But wait, there's more! 😉 As a valued subscriber, you'll also receive:
📚 Access to our exclusive pa

In [99]:
text = ""
for response in client.generate_stream(prompt, max_new_tokens=512):
    if not response.token.special:
        new_text = response.token.text
        print(new_text, end="")
        text += new_text

  Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive opportunity to subscribe to our top-notch paper supply for the next 1 year! 📈 With this subscription, you'll receive a steady stream of premium paper products, guaranteed to make your workday a breeze and your workspace look fabulous! 💪
Here's what you can expect from our Beet-y Awesome Paper Supply Subscription:
🌟 High-quality paper products, carefully selected to meet your every need.
📦 Regular deliveries of paper, so you'll never run out.
📊 A 10% discount on all paper purchases, just for subscribers! 💰
📝 A complimentary Dunder Mifflin pen, just for signing up! 🖋️
But wait, there's more! 😉 As a valued subscriber, you'll also receive:
📚 Access to our exclusive 

In [107]:
runpod.terminate_pod(pod["id"])

## References

- https://www.runpod.io/console/gpu-secure-cloud
- https://docs.runpod.io/docs/get-gpu-types
- https://github.com/facebookresearch/llama